In [ ]:
from data_loader import DataLoader, EELSLazyLoader
from filters import *
import matplotlib.pyplot as plt
import cv2
fp = '/home/somar/Desktop/own_stuff/cleaning script/example data files/SuperScan-MAADF_2025-02-26T153932.717062_2048x2048_0.ndata1'

ds = DataLoader(fp)

data = ds.raw_data
metadata = ds.raw_metadata

len(data), len(metadata)
img = data[0]
print(img.shape)
print("Size of the image:", img.size, img.size/2048)

# normalize to 0, 255
img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
print(img.min(), img.max())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def dgfilter(shape, params):
    sigma1, sigma2, weight = params
    s1 = sigma1 * sigma1
    s2 = sigma2 * sigma2

    yy_min = -shape[0] // 2
    yy_max = shape[0] // 2
    xx_min = -shape[1] // 2
    xx_max = shape[1] // 2
    yy, xx = np.meshgrid(np.linspace(yy_min, yy_max, shape[0]),
                         np.linspace(xx_min, xx_max, shape[1]),
                         indexing='ij')
    rr = np.square(xx / (shape[1] * 0.5)) + np.square(yy / (shape[0] * 0.5))
    return np.exp(-0.5 * rr / s1) - (1.0 - weight) * np.exp(-0.5 * rr / s2)

# Parameters to test
shapes = (256, 256)
parameter_sets = [
    (0.65, 0.2, 0.6),   # sigma1 < sigma2, balanced
    (0.3, 0.1, 0.5),   # sigma1 > sigma2, balanced
    (0.2, 0.4, 0.9),   # strong emphasis on first Gaussian
    (0.2, 0.4, 0.1),   # strong emphasis on second Gaussian
    (0.2, 0.2, 0.5),   # equal sigmas (degenerate case)
]

figs = []
for i, params in enumerate(parameter_sets):
    dg = dgfilter(shapes, params)
    fig, ax = plt.subplots()
    ax.imshow(dg, cmap='bwr', extent=[-1, 1, -1, 1])
    ax.set_title(f"σ₁={params[0]}, σ₂={params[1]}, weight={params[2]}")
    ax.set_xlabel("Normalized x")
    ax.set_ylabel("Normalized y")
    fig.colorbar(ax.images[0], ax=ax)
    figs.append(fig)

figs


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, VBox
from scipy.fft import fft2, ifft2, fftshift
from scipy.ndimage import gaussian_filter
import numpy as np
import matplotlib.pyplot as plt
from data_loader import DataLoader
filepath_ndata = '/home/somar/Desktop/own_stuff/cleaning script/example data files/081.ndata1'

img = DataLoader(filepath_ndata).raw_data[4]

def make_double_gaussian_filter(shape, sigma1, sigma2, weight):
    h, w = shape
    y, x = np.indices((h, w))
    cy, cx = h // 2, w // 2
    r2 = (x - cx)**2 + (y - cy)**2

    G1 = np.exp(-r2 / (2 * sigma1**2))
    G2 = np.exp(-r2 / (2 * sigma2**2))

    filter_kernel = G1 - (1 - weight) * G2
    return fftshift(filter_kernel)

# --- Apply filter ---
def apply_filter(img, sigma1, sigma2, weight):
    kernel = make_double_gaussian_filter(img.shape, sigma1, sigma2, weight)
    f_img = fft2(img)
    f_filtered = f_img * kernel
    filtered = np.real(ifft2(f_filtered))
    return filtered, kernel

# --- Interactive visualization ---
def interactive_filter(sigma1, sigma2, weight):
    filtered, kernel = apply_filter(img, sigma1, sigma2, weight)

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))

    ax[0].imshow(img, cmap='gray')
    ax[0].set_title("Original")

    ax[1].imshow(filtered, cmap='gray')
    ax[1].set_title(f"Filtered\n($\\sigma_1$={sigma1}, $\\sigma_2$={sigma2}, w={weight:.2f})")

    ax[2].imshow(kernel, cmap='viridis')
    ax[2].set_title("Filter (Freq Domain)")
    fig.colorbar(ax[2].images[0], ax=ax[2])
    for a in ax:
        a.axis('off')
    plt.tight_layout()
    plt.show()

# --- Create interactive sliders ---
interact(
    interactive_filter,
    sigma1=FloatSlider(min=1, max=100, step=1, value=20, description='σ1'),
    sigma2=FloatSlider(min=1, max=100, step=1, value=40, description='σ2'),
    weight=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.5, description='Weight')
)


In [ ]:
# %matplotlib qt5
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import cv2
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

different_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/different_sides.h5'

same_sides_irradiation = '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2525/26-03-2025/same_sides.h5'

from data_loader import DataLoader
img = DataLoader(same_sides_irradiation)


image = img.raw_data[24]
image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
image = image.astype(np.uint8)
image = image[590 : 590+ 100, 305 : 305 + 100]
thresh = cv2.inRange(image, 0, 3.1)  # 3.1 becomes 3 anyway
contours, _ = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
valid_contours = [cnt for cnt in contours if 400 <= cv2.contourArea(cnt) < 1e15]  # large upper bound
print(f"Total contours found: {len(valid_contours)}")

contour_mask = np.zeros_like(image, dtype=np.uint8)
cv2.drawContours(contour_mask, valid_contours, -1, color=255, thickness=cv2.FILLED)
image = image.astype(np.uint8)
masked_image = cv2.bitwise_and(image, image, mask=contour_mask)
thresh_1_Otsu = cv2.threshold(masked_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]


fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(masked_image, cmap='gray')
ax[0].set_title('Masked Image')
ax[1].imshow(thresh_1_Otsu, cmap='gray')
ax[1].set_title('Otsu Thresholding')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from data_loader import DataLoader

# Load image in grayscale
img = DataLoader(same_sides_irradiation)
img = img.raw_data[24]
img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
img = img.astype(np.uint8)
img = img[590 : 590 + 100, 305 : 305 + 100]

# Reshape image to a 2D array of pixels (needed for k-means)
pixel_values = img.reshape((-1, 1))
pixel_values = np.float32(pixel_values)  # Convert to float32 as required by cv2.kmeans

# Define criteria for k-means (max iterations or epsilon)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, 0.05)

# Number of clusters (2 for binary thresholding)
k = 4

# Apply k-means clustering
_, labels, centers = cv2.kmeans(pixel_values, k, None, criteria, 100, cv2.KMEANS_RANDOM_CENTERS)

# Convert centers to uint8
centers = np.uint8(centers)

# Map labels to corresponding center values
segmented_data = centers[labels.flatten()]
segmented_image = segmented_data.reshape(img.shape)

# Corrected thresholding step: flatten thresholded_img before indexing
thresholded_img = np.zeros_like(segmented_image).flatten()
foreground_cluster = np.argmax(centers)  # cluster with higher intensity
thresholded_img[labels.flatten() == foreground_cluster] = 255
thresholded_img = thresholded_img.reshape(img.shape)

# Show results
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.title('Original Image')
plt.imshow(img, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('Segmented Image (K-means)')
plt.imshow(segmented_image, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Thresholded Image')
plt.imshow(thresholded_img, cmap='gray')
plt.axis('off')

plt.show()

